In [1]:
from unsloth import FastVisionModel, FastLanguageModel
import os
import json
import pandas as pd
from PIL import Image
from tqdm import tqdm
import torch
from transformers import AutoProcessor, TextStreamer
import nltk
from datasets import load_from_disk

nltk.download("punkt")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: We'll be using `/tmp/unsloth_compiled_cache` for temporary Unsloth patches.
Standard import failed for UnslothCPOTrainer: No module named 'UnslothCPOTrainer'. Using tempfile instead!


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
import wandb

wandb.init(
    project="Prompting-Experiments",
    name="Prompt-Eval-Qwen2.5VL7B-Flickr30k",
    group="prompting-experiments",
    tags=["qwen", "image-captioning", "prompting", "flickr30k"],
    notes="Comparing handcrafted prompting strategies across multiple images and prompt types.",
    config={
        "model": "Qwen2.5-VL-7B",
        "prompting": "handcrafted",
        "num_prompts": 6,
        "num_runs_per_prompt": 2,
        "dataset": "flickr30k",
    },
)

In [4]:
model, tokenizer = FastVisionModel.from_pretrained(
    "unsloth/Qwen2-VL-7B-Instruct",
    load_in_4bit = True, # Use 4bit to reduce memory use. False for 16bit LoRA.
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
)

==((====))==  Unsloth 2025.5.3: Fast Qwen2_Vl patching. Transformers: 4.51.3.
   \\   /|    NVIDIA A40. Num GPUs = 1. Max memory: 44.448 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.6. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/6.85G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/572 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/392 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [5]:
model_id = "unsloth/Qwen2-VL-7B-Instruct"

model = FastVisionModel.for_inference(model)
processor = AutoProcessor.from_pretrained(model_id)

preprocessor_config.json:   0%|          | 0.00/572 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/4.33k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/392 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [6]:
test_dataset = load_from_disk("/workspace/data/filtered_dataset")

In [7]:
print("Filtered test Dataset:")
print(test_dataset)

print(f"Loaded {len(test_dataset)} image-caption entries\n")

print("📌 First 5 entries:")
first_5 = test_dataset.select(range(5))
for row in first_5:
    print(row, "\n")

print("📋 Column types:")
print(test_dataset.features)

print("\n🔎 Size")
print((len(test_dataset), len(test_dataset.features)))

Filtered test Dataset:
Dataset({
    features: ['image', 'caption', 'sentids', 'split', 'img_id', 'filename'],
    num_rows: 10
})
Loaded 10 image-caption entries

📌 First 5 entries:
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x375 at 0x758E12034430>, 'caption': ['Three Caucasian hikers in summer clothes walk a dirt path through a wooded meadow during the day.', 'Three people walking down a muddy dirt road on a cloudy day.', 'Three people are walking down a dirt road in the country.', 'Three people walking on a trail in the middle of the day.', 'Three people are walking down a dirt path.'], 'sentids': ['12500', '12501', '12502', '12503', '12504'], 'split': 'train', 'img_id': '2500', 'filename': '182169366.jpg'} 

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=356x500 at 0x758E12034550>, 'caption': ['A waitress in a black T-shirt and black shorts at Cafe Mambo serves a tray of beverages to a seated customer in a red tank top and black bandanna.'

In [8]:
from collections import Counter

filename_counts = Counter(test_dataset["filename"])

# Print filenames with more than 1 occurrence
for filename, count in filename_counts.items():
    if count > 1:
        print(f"{filename}: {count} occurrences")

num_duplicates = sum(1 for count in filename_counts.values() if count > 1)
print(f"\nTotal filenames with duplicates: {num_duplicates}")


Total filenames with duplicates: 0


In [12]:
import torch
import time
from transformers import TextStreamer
import pandas as pd

streamer = TextStreamer(tokenizer, skip_prompt=True)

def run_vlm_inference(prompt: str, filename: str, df: pd.DataFrame):
    """
    Perform inference on a given image (by filename) from the dataframe using a custom prompt.

    Args:
        prompt (str): The prompt text (can include mask tokens)
        filename (str): Filename of the image in the DataFrame
        df (pd.DataFrame): DataFrame with 'filename' and 'image' columns

    Returns:
        Tuple[str, float, float]: (Generated output, inference time in seconds, VRAM used in GB)
    """
    row = df[df["filename"] == filename]
    if row.empty:
        print(f"[ERROR] No image found with filename: {filename}")
        return None, 0.0, 0.0

    row = row.iloc[0]
    image = row["image"]
    if isinstance(image, dict) and "bytes" in image:
        from PIL import Image
        from io import BytesIO
        image = Image.open(BytesIO(image["bytes"])).convert("RGB")
    caption = row["caption"]

    messages = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {"type": "image", "image": image}
            ]
        }
    ]

    input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
    inputs = processor(images=image, text=input_text, return_tensors="pt").to("cuda")

    torch.cuda.reset_peak_memory_stats()
    start_mem = torch.cuda.memory_allocated() / 1024 / 1024 / 1024  # in GB
    start_time = time.time()

    print(f"🔹 Image: {filename}")
    print(f"🧾 Prompt: {prompt}")
    print("📤 Output:")

    # Perform inference
    output_ids = model.generate(
        **inputs,
        streamer=streamer,
        max_new_tokens=128,
        use_cache=True,
        temperature=1.5,
        min_p=0.1,
    )

    end_time = time.time()
    end_mem = torch.cuda.max_memory_allocated() / 1024**3

    time_taken = round(end_time - start_time, 3)
    vram_used = round(end_mem - start_mem, 3)
    decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    print(f"⏱️ Time taken: {time_taken} sec | 🧠 VRAM used: {vram_used} GB")
    print("-" * 80)

    return decoded_output, time_taken, vram_used

In [13]:
prompt_set_0 = ["",
              "How many people are playing soccer in this scene?",
               "An image of...",
               "You are a nature trail guide. Describe what’s happening in this scene to someone preparing for their first family hike.",
               "There is a <text_1> walking on a <text_2> in this image.",
               "Describe using format - Subject:  \_\_\_; Activity:  \_\_\_; Environment:  \_\_\_; Additional Notes: \_\_\_;"
               ]
prompt_set_1= ["",
              "How many people are playing soccer in this scene?",
               "An image of...",
               "You are a food critic observing customer service at cafes. Describe the interaction happening in this scene.",
               "A <text_1> is standing beside a <text_2> who is sitting down at a table.",
               "Describe using format - Subject:  \_\_\_; Activity:  \_\_\_; Environment:  \_\_\_; Additional Notes: \_\_\_;"
               ]
prompt_set_2= ["",
              "How many people are playing soccer in this scene?",
               "An image of...",
               "You are a weather reporter documenting children’s routines during rainy mornings. Describe what’s going on in this image.",
               "<text_1> are walking with <text_2> in the rain.",
               "Describe using format - Subject:  \_\_\_; Activity:  \_\_\_; Environment:  \_\_\_; Additional Notes: \_\_\_;"
               ]
prompt_set_3= ["",
              "How many people are playing soccer in this scene?",
               "An image of...",
               "You are a journalist covering a small-town parade. Describe the role and setting of the clown in this festive scene.",
               "A <text_1> with bright makeup and colorful clothes stands near a <text_2> during an event.",
               "Describe using format - Subject:  \_\_\_; Activity:  \_\_\_; Environment:  \_\_\_; Additional Notes: \_\_\_;"
               ]
prompt_set_4= ["",
              "How many people are playing soccer in this scene?",
               "An image of...",
               "You are a winter sports photographer recounting a joyful moment you captured. Describe the scene with vivid detail.",
               "Two <text_1> are sitting on a snowy slope with their arms <text_2>.",
               "Describe using format - Subject:  \_\_\_; Activity:  \_\_\_; Environment:  \_\_\_; Additional Notes: \_\_\_;"
               ]
prompt_set_5= ["",
              "How many people are playing soccer in this scene?",
               "An image of...",
               "You are a market researcher documenting traditional grocery stores. Describe what the boy is doing and what the store looks like.",
               "A <text_1> is behind a counter filled with <text_2> in a small shop.",
               "Describe using format - Subject:  \_\_\_; Activity:  \_\_\_; Environment:  \_\_\_; Additional Notes: \_\_\_;"
               ]
prompt_set_6= ["",
              "How many people are playing soccer in this scene?",
               "An image of...",
               "You are an IT workplace culture analyst. Describe the work environment and team dynamic shown in this image.",
               "A <text_1> is using a laptop at a desk while two <text_2> are in the background.",
               "Describe using format - Subject:  \_\_\_; Activity:  \_\_\_; Environment:  \_\_\_; Additional Notes: \_\_\_;"
               ]
prompt_set_7= ["",
              "How many people are playing soccer in this scene?",
               "An image of...",
               "You are a canine trainer assessing a working dog’s field behavior. Describe the dog’s posture and role based on the scene..",
               "A <text_1> is wearing a vest and holding a <text_2> in its mouth.",
               "Describe using format - Subject:  \_\_\_; Activity:  \_\_\_; Environment:  \_\_\_; Additional Notes: \_\_\_;"
               ]
prompt_set_8= ["",
              "How many people are playing soccer in this scene?",
               "An image of...",
               "You are a cultural anthropologist documenting traditional street labor. Describe what the person is doing and how they are carrying the items.",
               "A <text_1> is walking while carrying <text_2> suspended from a pole.",
               "Describe using format - Subject:  \_\_\_; Activity:  \_\_\_; Environment:  \_\_\_; Additional Notes: \_\_\_;"
               ]
prompt_set_9= ["",
              "How many people are playing soccer in this scene?",
               "An image of...",
               "You are a museum docent giving a tour. Describe the activity of the woman in the context of the art gallery.",
               "A <text_1> is painting a replica of an artwork in front of <text_2>.",
               "Describe using format - Subject:  \_\_\_; Activity:  \_\_\_; Environment:  \_\_\_; Additional Notes: \_\_\_;"
               ]
PROMPT_LIST = [prompt_set_0, prompt_set_1, prompt_set_2, prompt_set_3, prompt_set_4, prompt_set_5, prompt_set_6, prompt_set_7, prompt_set_8, prompt_set_9]

In [14]:
test_img_ids = [2500, 2501, 2502, 2503, 2504, 2505, 2506, 2507, 2508, 2509]
df = test_dataset.to_pandas()

all_outputs = {}
all_times = {}
all_vram = {}

# Loop through each image and its corresponding prompt set
for i, img_id in enumerate(test_img_ids):
    row = test_dataset.filter(lambda x: x["img_id"] == str(img_id))
    if len(row) == 0:
        print(f"[WARNING] No image found with img_id {img_id}")
        continue

    filename = row[0]["filename"]
    prompts = PROMPT_LIST[i]

    print(f"\n🔍 Running prompts for image: {filename} (img_id: {img_id})\n" + "-"*60)

    inference_outputs = []
    inference_times = []
    vram_usages = []

    for prompt in prompts:
        # Run the same prompt twice for the same image
        for run in range(2):
            print(f"🔁 Run {run+1} for prompt: {prompt[:50]}{'...' if len(prompt) > 50 else ''}")
            output, time_taken, vram_used = run_vlm_inference(prompt, filename, df=df)
            inference_outputs.append(output)
            inference_times.append(time_taken)
            vram_usages.append(vram_used)

    # Save to overall dicts
    all_outputs[img_id] = inference_outputs
    all_times[img_id] = inference_times
    all_vram[img_id] = vram_usages



🔍 Running prompts for image: 182169366.jpg (img_id: 2500)
------------------------------------------------------------
🔁 Run 1 for prompt: 
🔹 Image: 182169366.jpg
🧾 Prompt: 
📤 Output:
<|object_ref_start|><|object_ref_start|><|object_ref_start|><|object_ref_start|><|object_ref_start|><|object_ref_start|><|object_ref_start|><|object_ref_start|><|object_ref_start|><|object_ref_start|><|object_ref_start|><|object_ref_start|><|object_ref_start|><|object_ref_start|><|object_ref_start|><|box_start|>(625,312),(735,691)<|box_end|><|im_end|>
⏱️ Time taken: 1.674 sec | 🧠 VRAM used: 0.191 GB
--------------------------------------------------------------------------------
🔁 Run 2 for prompt: 
🔹 Image: 182169366.jpg
🧾 Prompt: 
📤 Output:
<|object_ref_start|><|object_ref_start|><|object_ref_start|><|object_ref_start|><|object_ref_start|><|object_ref_start|><|object_ref_start|><|object_ref_start|><|object_ref_start|><|object_ref_start|><|object_ref_start|><|object_ref_start|><|object_ref_start|><|obje

In [ ]:
for img_id in all_outputs:
    print(f"\n🖼️ Image ID: {img_id}")
    print("-" * 60)
    for i, (output, time_taken, vram_used) in enumerate(zip(all_outputs[img_id], all_times[img_id], all_vram[img_id])):
        print(f"🔹 Prompt {i+1}")
        print(f"📝 Output: {output}")
        print(f"⏱️ Inference Time: {time_taken:.3f} sec")
        print(f"💾 VRAM Used: {vram_used:.3f} GB")
        print("-" * 60)

In [15]:
from sentence_transformers import SentenceTransformer, util
from pycocoevalcap.cider.cider import Cider
from pycocoevalcap.spice.spice import Spice
from pycocoevalcap.tokenizer.ptbtokenizer import PTBTokenizer
import pandas as pd

sbert_model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
def compute_cosine_similarity(reference_captions, generated_caption):
    try:
        total_score = 0.0
        for caption in reference_captions:
            ref_embed = sbert_model.encode(caption, convert_to_tensor=True)
            gen_embed = sbert_model.encode(generated_caption, convert_to_tensor=True)
            score = util.cos_sim(gen_embed, ref_embed).item()
            total_score += score
        avg_score = total_score / len(reference_captions) if reference_captions else 0.0
        return avg_score
    except Exception as e:
        print(f"Error computing cosine similarity: {e}")
        return 0.0

In [17]:
def compute_cider_spice_scores(reference_caption, generated_caption):
    refs = {0: [reference_caption if reference_caption else ""]}
    hypos = {0: [generated_caption if generated_caption else ""]}

    # print(f"Generated caption: {generated_caption}")
    # print(f"Generated hypos: {hypos}")

    ptb = PTBTokenizer()
    refs_tok = ptb.tokenize({i: [{"caption": c} for c in caps] for i, caps in refs.items()})
    hypos_tok = ptb.tokenize({i: [{"caption": hypos[i][0]}] for i in hypos})

    all_scores = {}

    for scorer, name in [(Cider(), "CIDEr"), (Spice(), "SPICE")]:
        try:
            avg_score, _ = scorer.compute_score(refs_tok, hypos_tok)
            if name == "SPICE":
                # SPICE returns dicts per image
                all_scores[name] = avg_score.get("All", {}).get("f", 0.0) if isinstance(avg_score, dict) else avg_score
            else:
                all_scores[name] = avg_score
        except Exception as e:
            print(f"[ERROR] {name} scoring failed: {e}")
            all_scores[name] = 0.0

    return all_scores

In [18]:
from cider import Cider

PICKLE_PATH = "/workspace/data/flickr-df.p"

def compute_cider2_score(reference_caption, generated_caption):
    refs = {"0": reference_caption if isinstance(reference_caption, list) else [reference_caption]}
    hypos = [{"image_id": "0", "caption": [generated_caption]}]  # Fix: caption should be a list

    cider = Cider()
    score, _ = cider.compute_score(refs, hypos, PICKLE_PATH)

    return score

In [19]:
def evaluate_all_metrics(reference_caption, generated_caption):
    cider_spice_scores = compute_cider_spice_scores(reference_caption, generated_caption)
    cosine_sim = compute_cosine_similarity([reference_caption], generated_caption)
    cider2 = compute_cider2_score(reference_caption, generated_caption)

    return {
        "cosine_similarity": round(cosine_sim, 4),
        #"CIDEr": round(cider_spice_scores.get("CIDEr", 0.0), 4),
        "SPICE": round(cider_spice_scores.get("SPICE", 0.0), 4),
        "CIDEr": round(cider2, 4)
    }

In [20]:
all_metrics_scores = {}

for img_id, outputs in all_outputs.items():
    row = test_dataset.filter(lambda x: x["img_id"] == str(img_id))
    if len(row) == 0:
        print(f"[WARNING] No ground truth found for img_id {img_id}")
        continue

    ground_truths = row[0]["caption"]  # This is a list of 5 captions
    print(f"\n🔍 Evaluating image ID: {img_id} | Ground truths: {len(ground_truths)} captions\n{'-'*80}")
    
    scores_for_image = []

    for idx, gen_output in enumerate(outputs):  # 12 outputs (6 prompts x 2 runs)
        cider_total, spice_total, cos_total = 0.0, 0.0, 0.0

        for ref in ground_truths:
            metrics = evaluate_all_metrics(ref, gen_output)
            cider_total += metrics["CIDEr"]
            spice_total += metrics["SPICE"]
            cos_total += metrics["cosine_similarity"]

        n_refs = len(ground_truths)
        avg_metrics = {
            "CIDEr": round(cider_total / n_refs, 4),
            "SPICE": round(spice_total / n_refs, 4),
            "cosine_similarity": round(cos_total / n_refs, 4)
        }

        print(f"📝 Prompt Run {idx+1} | CIDEr: {avg_metrics['CIDEr']} | SPICE: {avg_metrics['SPICE']} | CosSim: {avg_metrics['cosine_similarity']}")
        scores_for_image.append(avg_metrics)

    all_metrics_scores[img_id] = scores_for_image


🔍 Evaluating image ID: 2500 | Ground truths: 5 captions
--------------------------------------------------------------------------------


PTBTokenizer tokenized 18 tokens at 244.59 tokens per second.
PTBTokenizer tokenized 16 tokens at 428.91 tokens per second.


Progress: 384.5M / 384.5M (100.0%)
Extracting stanford-corenlp-3.6.0 ...
Done.
[ERROR] CIDEr scoring failed: Cider.compute_score() missing 1 required positional argument: 'pfile_path'


Parsing reference captions
Initiating Stanford parsing pipeline
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.TokenizerAnnotator - TokenizerAnnotator: No tokenizer type provided. Defaulting to PTBTokenizer.
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator parse
[main] INFO edu.stanford.nlp.parser.common.ParserGrammar - Loading parser from serialized file edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz ... 
done [0.4 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
Loading classifier from edu/stanford/nlp/models/ner/english.all.3class.distsim.crf.ser.gz ... done [1.2 sec].
Loading classifier from edu/stanford/nlp/models/ner/english.muc.7class.distsim.crf.ser.gz ... done [0.5 sec].
Loading classif

SPICE evaluation took: 7.110 s


...

SPICE evaluation took: 687.0 ms


PTBTokenizer tokenized 12 tokens at 277.08 tokens per second.
PTBTokenizer tokenized 89 tokens at 2181.88 tokens per second.


[ERROR] CIDEr scoring failed: Cider.compute_score() missing 1 required positional argument: 'pfile_path'


Parsing reference captions
Parsing test captions


SPICE evaluation took: 704.6 ms
📝 Prompt Run 12 | CIDEr: 2.6653 | SPICE: 0.0753 | CosSim: 0.2895


In [21]:
import pandas as pd

for img_id, scores in all_metrics_scores.items():
    print(f"\n📸 Image ID: {img_id}")
    print("-" * 60)

    df_temp = pd.DataFrame(scores)
    prompt_labels = []
    for i in range(len(scores)):
        prompt_num = (i // 2) + 1  # Prompts 1–6
        run_num = (i % 2) + 1      # Run 1 or Run 2
        prompt_labels.append(f"Prompt {prompt_num} (Run {run_num})")

    df_temp.index = prompt_labels
    print(df_temp.round(4))
    print("=" * 60)



📸 Image ID: 2500
------------------------------------------------------------
                   CIDEr   SPICE  cosine_similarity
Prompt 1 (Run 1)  1.7138  0.0000             0.0060
Prompt 1 (Run 2)  1.7138  0.0000             0.0060
Prompt 2 (Run 1)  2.4246  0.0715             0.0926
Prompt 2 (Run 2)  2.4246  0.0715             0.0926
Prompt 3 (Run 1)  1.2119  0.0000             0.0631
Prompt 3 (Run 2)  1.2119  0.0000             0.0631
Prompt 4 (Run 1)  3.9525  0.0372             0.4210
Prompt 4 (Run 2)  3.9525  0.0372             0.4210
Prompt 5 (Run 1)  3.7612  0.0143             0.1626
Prompt 5 (Run 2)  3.7612  0.0143             0.1626
Prompt 6 (Run 1)  2.3237  0.0631             0.2637
Prompt 6 (Run 2)  2.3237  0.0631             0.2637

📸 Image ID: 2501
------------------------------------------------------------
                   CIDEr   SPICE  cosine_similarity
Prompt 1 (Run 1)  1.4644  0.0216             0.2534
Prompt 1 (Run 2)  1.4644  0.0216             0.2534
Prompt 2 (

In [24]:
# Log results table
table = wandb.Table(columns=["img_id", "prompt_type", "run", "image", "ground_truth", "prediction"])

for img_id in all_outputs:
    filename_row = df[df["img_id"].astype(str) == str(img_id)]
    if filename_row.empty:
        continue

    filename = filename_row.iloc[0]["filename"]
    gt_captions = filename_row.iloc[0]["caption"]
    image_row = test_dataset.filter(lambda x: x["img_id"] == str(img_id))
    if len(image_row) == 0:
        continue
    image_pil = image_row[0]["image"]  # This is already PIL.Image

    for prompt_index in range(6):
        for run in range(2):
            row_idx = prompt_index * 2 + run
            prediction = all_outputs[img_id][row_idx] if row_idx < len(all_outputs[img_id]) else "N/A"

            table.add_data(
                str(img_id),
                f"Prompt {prompt_index + 1}",
                f"Run {run + 1}",
                wandb.Image(image_pil),
                gt_captions[0],  # only logging first GT
                prediction
            )

wandb.log({"Prompting Comparison Results": table})

# Log metrics for each prompt run
for img_id, scores in all_metrics_scores.items():
    for i, score in enumerate(scores):
        wandb.log({
            f"{img_id}/Prompt {i//2 + 1}/Run {i%2 + 1}/CIDEr": score["CIDEr"],
            f"{img_id}/Prompt {i//2 + 1}/Run {i%2 + 1}/SPICE": score["SPICE"],
            f"{img_id}/Prompt {i//2 + 1}/Run {i%2 + 1}/Cosine": score["cosine_similarity"]
        })

In [25]:
def log_prompt_metrics_to_excel(
    img_id: str,
    model_name: str,
    inference_outputs: list,
    metrics: list,
    inference_times: list,
    vram_usages: list,
    df: pd.DataFrame,
    test_dataset,
    output_excel_path: str = "prompt_tuning_results_flickr_qwen.xlsx"
):
    from openpyxl import Workbook, load_workbook
    from openpyxl.drawing.image import Image as ExcelImage
    from openpyxl.styles import Alignment
    from PIL import Image as PILImage
    import os

    # Extract from test_dataset using img_id
    row = test_dataset.filter(lambda x: x["img_id"] == str(img_id))[0]
    filename = row["filename"]
    image_path = os.path.join("/workspace/data/filtered_dataset", filename)
    original_caption = row["caption"]
    original_caption_text = "\n".join(original_caption) if isinstance(original_caption, list) else str(original_caption)

    # Load or create Excel file
    if os.path.exists(output_excel_path):
        wb = load_workbook(output_excel_path)
        ws = wb.active
    else:
        wb = Workbook()
        ws = wb.active
        ws.title = "Prompt Evaluation"
        headers = [
            "Model", "Image ID", "Image", "Original Captions",
            "Prompt", "Output", "Inference Time (s)", "VRAM Used (GB)",
            "CIDEr", "SPICE", "Cosine Similarity"
        ]
        ws.append(headers)

    start_row = ws.max_row + 1

    # Write both runs (2x6 = 12 rows)
        # Write both runs (2x6 = 12 rows) — run-wise grouping
    for run in range(2):  # 0 = Run 1, 1 = Run 2
        row_start = start_row + run * 6
        for i in range(6):  # Prompt 1–6
            idx = i * 2 + run  # FIXED: Correct index mapping
            ws.append([
                model_name,
                img_id,
                "",
                original_caption_text,
                f"Prompt {i+1} (Run {run+1})",
                inference_outputs[idx],
                inference_times[idx],
                vram_usages[idx],
                metrics[idx]["CIDEr"],
                metrics[idx]["SPICE"],
                metrics[idx]["cosine_similarity"]
            ])

        # Merge columns A–D separately for each run
        for col in ["A", "B", "C", "D"]:
            ws.merge_cells(f"{col}{row_start}:{col}{row_start + 5}")

    # Apply formatting
    align_top_wrap = Alignment(wrap_text=True, vertical="top")
    align_top = Alignment(vertical="top")

    for row in range(start_row, start_row + 12):
        for col_letter in ["A", "B", "C", "E"]:
            ws[f"{col_letter}{row}"].alignment = align_top
        for col_letter in ["D", "F"]:
            ws[f"{col_letter}{row}"].alignment = align_top_wrap
        ws.row_dimensions[row].height = 120

    # Set column widths
    ws.column_dimensions["D"].width = 50  # Captions
    ws.column_dimensions["F"].width = 50  # Output

    # Insert image once (in first run)
        # Embed image in first run row
    if os.path.exists(image_path):
        try:
            print(f"[INFO] Embedding image from path: {image_path}")
            pil_img = PILImage.open(image_path)
            pil_img.thumbnail((300, 300))  # Resize for better fit

            excel_img = ExcelImage(image_path)
            excel_img.width = 150
            excel_img.height = 150
            excel_img.anchor = f"C{start_row}"  # Image embedded in first run row
            ws.add_image(excel_img)
        except Exception as e:
            print(f"[ERROR] Could not embed image for {img_id}: {e}")
    else:
        print(f"[WARNING] Image path does not exist: {image_path}")

    wb.save(output_excel_path)
    print(f"✅ Logged both runs for image {img_id} to {output_excel_path}")

In [26]:
for img_id in all_outputs.keys():
    log_prompt_metrics_to_excel(
        img_id=img_id,
        model_name=model_id,
        inference_outputs=all_outputs[img_id],
        metrics=all_metrics_scores[img_id],
        inference_times=all_times[img_id],
        vram_usages=all_vram[img_id],
        df=df,
        test_dataset=test_dataset,
        output_excel_path="prompt_tuning_results_flickr_qwen.xlsx"
    )

Filter:   0%|          | 0/10 [00:00<?, ? examples/s]

[INFO] Embedding image from path: /workspace/data/filtered_dataset/182169366.jpg
✅ Logged both runs for image 2500 to prompt_tuning_results_flickr_qwen.xlsx


Filter:   0%|          | 0/10 [00:00<?, ? examples/s]

[INFO] Embedding image from path: /workspace/data/filtered_dataset/182184279.jpg
✅ Logged both runs for image 2501 to prompt_tuning_results_flickr_qwen.xlsx


Filter:   0%|          | 0/10 [00:00<?, ? examples/s]

...

[INFO] Embedding image from path: /workspace/data/filtered_dataset/1827560917.jpg
✅ Logged both runs for image 2507 to prompt_tuning_results_flickr_qwen.xlsx


Filter:   0%|          | 0/10 [00:00<?, ? examples/s]

[INFO] Embedding image from path: /workspace/data/filtered_dataset/182825639.jpg
✅ Logged both runs for image 2508 to prompt_tuning_results_flickr_qwen.xlsx


Filter:   0%|          | 0/10 [00:00<?, ? examples/s]

[INFO] Embedding image from path: /workspace/data/filtered_dataset/18284727.jpg
✅ Logged both runs for image 2509 to prompt_tuning_results_flickr_qwen.xlsx
